In [28]:
## GENERATING CAPTIONS FOR PROVIDED IMAGES

In [42]:
import os
import pandas as pd
import string
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import cv2
from skimage import exposure
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, add
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, concatenate, RepeatVector

In [30]:
##Checking the txt file of generated caption

In [24]:
# Path to the generated text file
file_path = "training_captions.txt"

# Open the file in read mode
with open(file_path, "r") as file:
    # Read the content of the file
    content = file.read()

# Display the content
print(content)


Image: img1.jpg
Caption: Alai Darwaza, the southern gateway of the Quwwat-ul-Islam Mosque in the Qutb complex, Delhi, India, is a masterpiece of Indo-Islamic architecture. Built in 1311 by Alauddin Khalji, the gateway features intricate carvings, beautiful calligraphy, and exquisite inlay work in white marble and red sandstone. The Alai Darwaza is renowned for its horseshoe-shaped arches, elaborate lattice screens, and geometric patterns that reflect the architectural innovations introduced during the Khalji dynasty. Visitors can explore the gateway and the surrounding structures in the Qutb complex, which includes the famous Qutub Minar, the tallest brick minaret in the world. The Alai Darwaza stands as a testament to the rich cultural heritage and architectural brilliance of medieval India.

Image: img10.jpg
Caption: Alai Darwaza, the southern gateway of the Quwwat-ul-Islam Mosque in the Qutb complex, Delhi, India, is a masterpiece of Indo-Islamic architecture. Built in 1311 by Alaud

In [3]:
# Path to the generated text file
file_path = "testing_captions.txt"

# Open the file in read mode
with open(file_path, "r") as file:
    # Read the content of the file
    content = file.read()

# Display the content
print(content)

Image: img175.jpg
Caption: Alai Darwaza, the southern gateway of the Quwwat-ul-Islam Mosque in the Qutb complex, Delhi, India, is a masterpiece of Indo-Islamic architecture. Built in 1311 by Alauddin Khalji, the gateway features intricate carvings, beautiful calligraphy, and exquisite inlay work in white marble and red sandstone. The Alai Darwaza is renowned for its horseshoe-shaped arches, elaborate lattice screens, and geometric patterns that reflect the architectural innovations introduced during the Khalji dynasty. Visitors can explore the gateway and the surrounding structures in the Qutb complex, which includes the famous Qutub Minar, the tallest brick minaret in the world. The Alai Darwaza stands as a testament to the rich cultural heritage and architectural brilliance of medieval India.

Image: img180.jpg
Caption: Alai Darwaza, the southern gateway of the Quwwat-ul-Islam Mosque in the Qutb complex, Delhi, India, is a masterpiece of Indo-Islamic architecture. Built in 1311 by Al

In [54]:
# Split the content into lines
lines = content.split("\n")

# Create a list to store rows of data
data = []

# Parse each line and extract image filename and caption
for line in lines:
    if line.strip():  # Check if the line is not empty
        filename, caption = line.split(":", 1)  # Split by colon (:) to separate filename and caption
        data.append((filename.strip(), caption.strip()))  # Append the tuple (filename, caption) to the data list

# Create a DataFrame from the data
df = pd.DataFrame(data, columns=["Image Filename", "Caption"])

# Display the DataFrame
print(df)


    Image Filename                                            Caption
0            Image                                           img1.jpg
1          Caption  Alai Darwaza, the southern gateway of the Quww...
2            Image                                          img10.jpg
3          Caption  Alai Darwaza, the southern gateway of the Quww...
4            Image                                         img101.jpg
..             ...                                                ...
655        Caption  Khajuraho, located in Madhya Pradesh, India, i...
656          Image                                           img8.jpg
657        Caption  Khajuraho, located in Madhya Pradesh, India, i...
658          Image                                           img9.jpg
659        Caption  Khajuraho, located in Madhya Pradesh, India, i...

[660 rows x 2 columns]


In [31]:
##Preprocessing Captions

In [36]:
# Function to preprocess captions from text files
def preprocess_captions_from_files(folder):
    captions = []
    for filename in os.listdir(folder):
        if filename.endswith(".txt"):
            with open(os.path.join(folder, filename), "r") as file:
                caption = file.read().strip().lower()  # Convert to lowercase
                caption = caption.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
                captions.append(caption)
    return captions


# Example paths to caption folders
training_caption_folder = r"C:\Users\admin\Downloads\archive\Indian-monuments\images\train"
testing_caption_folder = r"C:\Users\admin\Downloads\archive\Indian-monuments\images\test"

# Preprocess captions for training and testing data
training_captions = preprocess_captions_from_files(training_caption_folder)
testing_captions = preprocess_captions_from_files(testing_caption_folder)

# Tokenize and pad captions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(training_captions)

training_sequences = tokenizer.texts_to_sequences(training_captions)
testing_sequences = tokenizer.texts_to_sequences(testing_captions)

max_caption_length = max(max(len(seq) for seq in training_sequences), max(len(seq) for seq in testing_sequences))

padded_training_sequences = pad_sequences(training_sequences, maxlen=max_caption_length, padding='post')
padded_testing_sequences = pad_sequences(testing_sequences, maxlen=max_caption_length, padding='post')

In [37]:
# Print original and preprocessed captions for training data
print("Original captions for training data:")
for caption in training_captions:
    print(caption)

print("\nPreprocessed captions for training data:")
for sequence in padded_training_sequences:
    print(tokenizer.sequences_to_texts([sequence])[0])

# Print original and preprocessed captions for testing data
print("\nOriginal captions for testing data:")
for caption in testing_captions:
    print(caption)

print("\nPreprocessed captions for testing data:")
for sequence in padded_testing_sequences:
    print(tokenizer.sequences_to_texts([sequence])[0])


Original captions for training data:
ajanta caves located in maharashtra india are a series of 29 rockcut cave monuments created between the 2nd century bce and the 6th century ce these caves were used as a monastic retreat for buddhist monks and are renowned for their stunning murals and sculptures depicting the jataka tales which narrate the previous lives of the buddha the caves are nestled in a panoramic gorge and were rediscovered in 1819 by a british officer the most notable caves include cave 1 with its impressive bodhisattva murals and cave 26 featuring a massive reclining buddha visitors can take guided tours to explore the intricate carvings and paintings that offer insights into ancient buddhist art and teachings the site is a unesco world heritage site and a testament to the architectural and artistic achievements of ancient india
alai darwaza the southern gateway of the quwwatulislam mosque in the qutb complex delhi india is a masterpiece of indoislamic architecture built 

In [32]:
###Checking on the images in respective folder

In [1]:
# Define the path
path = r"C:\Users\admin\Downloads\archive\Indian-monuments\Images"

# Walk through the directory and list the number of files
for dirpath, dirnames, filenames in os.walk(path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")


There are 2 directories and 0 images in 'C:\Users\admin\Downloads\archive\Indian-monuments\Images'.
There are 24 directories and 0 images in 'C:\Users\admin\Downloads\archive\Indian-monuments\Images\test'.
There are 0 directories and 31 images in 'C:\Users\admin\Downloads\archive\Indian-monuments\Images\test\Ajanta Caves'.
There are 0 directories and 36 images in 'C:\Users\admin\Downloads\archive\Indian-monuments\Images\test\alai_darwaza'.
There are 0 directories and 35 images in 'C:\Users\admin\Downloads\archive\Indian-monuments\Images\test\alai_minar'.
There are 0 directories and 30 images in 'C:\Users\admin\Downloads\archive\Indian-monuments\Images\test\basilica_of_bom_jesus'.
There are 0 directories and 34 images in 'C:\Users\admin\Downloads\archive\Indian-monuments\Images\test\Charar-E- Sharif'.
There are 0 directories and 40 images in 'C:\Users\admin\Downloads\archive\Indian-monuments\Images\test\charminar'.
There are 0 directories and 30 images in 'C:\Users\admin\Downloads\archi

In [33]:
####Preprocessing Images

In [2]:
# Define the input and output directories
input_root = r"C:\Users\admin\Downloads\archive\Indian-monuments\Images"
output_root = r"C:\Users\admin\Downloads\archive\Indian-monuments\Preprocessed_Images"

# Create the output directory if it doesn't exist
if not os.path.exists(output_root):
    os.makedirs(output_root)

# Walk through the directory structure
for root, dirs, files in os.walk(input_root):
    for filename in files:
        if filename.endswith((".jpg", ".jpeg", ".png")):
            # Read the image
            image_path = os.path.join(root, filename)
            image = cv2.imread(image_path)
            
            if image is not None:
                # Perform preprocessing 
                resized_image = cv2.resize(image, (224, 224))
                normalized_image = resized_image.astype(float) / 255.0  # Normalize pixel values
                
                # Define the output directory structure
                relative_path = os.path.relpath(root, input_root)
                output_folder = os.path.join(output_root, relative_path)
                output_path = os.path.join(output_folder, filename)
                
                # Create the output directory if it doesn't exist
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)
                
                # Save the preprocessed image
                cv2.imwrite(output_path, normalized_image * 255)


In [11]:
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import chardet

def load_captions(caption_file):
    with open(caption_file, 'rb') as f:
        raw_data = f.read()
        encoding = chardet.detect(raw_data)['encoding']
    
    try:
        with open(caption_file, 'r', encoding=encoding) as f:
            captions = f.readlines()
    except UnicodeDecodeError:
        print(f"UnicodeDecodeError: Failed to decode {caption_file} with detected encoding {encoding}.")
        return None
    captions = [caption.strip().split('\t') for caption in captions]
    return captions


# Paths to your dataset
image_dir_train = r"C:\Users\admin\Downloads\archive\Indian-monuments\Images\train"
image_dir_test = r"C:\Users\admin\Downloads\archive\Indian-monuments\Images\test"
caption_file_train = r"C:\Users\admin\Downloads\archive\Indian-monuments\captions\captions_train.txt"
caption_file_test = r"C:\Users\admin\Downloads\archive\Indian-monuments\captions\captions_test.txt"

# Load captions for training and testing
captions_train = load_captions(caption_file_train)
captions_test = load_captions(caption_file_test)

# Extract image paths from the train and test folders
train_image_paths = []
test_image_paths = []

for root, dirs, files in os.walk(image_dir_train):
    for file in files:
        if file.endswith('.jpg'):  # Adjust file extension as per your images
            train_image_paths.append(os.path.join(root, file))

for root, dirs, files in os.walk(image_dir_test):
    for file in files:
        if file.endswith('.jpg'):  # Adjust file extension as per your images
            test_image_paths.append(os.path.join(root, file))

# Tokenize captions
VOCAB_SIZE = 5000  # Adjust based on dataset
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<unk>", filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts([caption[0] for caption in captions_train + captions_test])

# Convert captions to sequences for training and testing
caption_seqs_train = tokenizer.texts_to_sequences([caption[0] for caption in captions_train])
caption_seqs_test = tokenizer.texts_to_sequences([caption[0] for caption in captions_test])

# Find maximum caption length
max_caption_length_train = max(len(seq) for seq in caption_seqs_train)
max_caption_length_test = max(len(seq) for seq in caption_seqs_test)
max_caption_length = max(max_caption_length_train, max_caption_length_test)

# Pad sequences for training and testing
caption_seqs_train = pad_sequences(caption_seqs_train, maxlen=max_caption_length, padding='post')
caption_seqs_test = pad_sequences(caption_seqs_test, maxlen=max_caption_length, padding='post')

# Example of how to use the tokenizer and sequences
print("Vocabulary size:", len(tokenizer.word_index))
print("Max caption length for training:", max_caption_length_train)
print("Max caption length for testing:", max_caption_length_test)

# Print example captions and their sequences for training and testing
print("Training captions and sequences:")
for i in range(5):  # Print first 5 captions and their sequences for training
    print("Caption:", captions_train[i][0])
    print("Sequence:", caption_seqs_train[i])
    print()

print("Testing captions and sequences:")
for i in range(5):  # Print first 5 captions and their sequences for testing
    print("Caption:", captions_test[i][0])
    print("Sequence:", caption_seqs_test[i])
    print()

# Example paths for training and testing images
print("Number of training images:", len(train_image_paths))
print("Number of testing images:", len(test_image_paths))


Vocabulary size: 883
Max caption length for training: 16
Max caption length for testing: 15
Training captions and sequences:
Caption: C:\Users\admin\Downloads\archive\Indian-monuments\Images\train\qutub_minar\img1.jpg
Sequence: [  3   4   5   6   7   8   9   2  11  16  13 184  10   0   0   0]

Caption: C:\Users\admin\Downloads\archive\Indian-monuments\Images\train\qutub_minar\img10.jpg
Sequence: [  3   4   5   6   7   8   9   2  11  16  13 165  10   0   0   0]

Caption: C:\Users\admin\Downloads\archive\Indian-monuments\Images\train\qutub_minar\img100.jpg
Sequence: [  3   4   5   6   7   8   9   2  11  16  13 312  10   0   0   0]

Caption: C:\Users\admin\Downloads\archive\Indian-monuments\Images\train\qutub_minar\img101.jpg
Sequence: [  3   4   5   6   7   8   9   2  11  16  13 286  10   0   0   0]

Caption: C:\Users\admin\Downloads\archive\Indian-monuments\Images\train\qutub_minar\img102.jpg
Sequence: [  3   4   5   6   7   8   9   2  11  16  13 287  10   0   0   0]

Testing captions a

In [ ]:
#Model

In [3]:
# Build InceptionV3 encoder
def build_encoder():
    base_model = InceptionV3(weights='imagenet')
    encoder = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)
    return encoder

# Build LSTM-based decoder
def build_decoder(vocab_size, embedding_dim, units, max_length):
    # Image feature placeholder
    image_features = Input(shape=(2048,), name='image_features')

    # Captions as sequences
    inputs = Input(shape=(max_length,), name='input_captions')
    embeddings = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(inputs)
    embeddings = Dropout(0.5)(embeddings)

    # Repeat image features to match max_length
    image_features_repeated = RepeatVector(max_length)(image_features)

    # Merge image features and caption embeddings
    merged = concatenate([image_features_repeated, embeddings])
    lstm = LSTM(units, return_sequences=True)(merged)
    lstm = Dropout(0.5)(lstm)
    outputs = Dense(vocab_size, activation='softmax')(lstm)

    model = Model(inputs=[image_features, inputs], outputs=outputs)
    return model

In [4]:
# Build full image captioning model
def build_image_captioning_model(encoder, decoder):
    image_input = Input(shape=(299, 299, 3), name='image_input')
    encoded_image = encoder(image_input)
    caption_input = Input(shape=(MAX_LENGTH,), name='caption_input')
    output = decoder([encoded_image, caption_input])
    
    model = Model(inputs=[image_input, caption_input], outputs=output)
    return model

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define constants
VOCAB_SIZE = 813  # You can adjust this value as needed
MAX_LENGTH = 143     # You can adjust this value as needed

# Paths
image_dir_train = r'C:\Users\admin\Downloads\archive\Indian-monuments\Images\train'
caption_file = r"C:\Users\admin\Downloads\archive\Indian-monuments\captions\captions_train.txt"

# Function to load captions (assuming you have this function implemented)
def load_captions(filepath):
    with open(filepath, 'r') as file:
        captions = file.readlines()
    return [line.strip().split('\t') for line in captions]

# Load captions
captions = load_captions(caption_file)

# Tokenize captions
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<unk>")
tokenizer.fit_on_texts([caption[0] for caption in captions])

# Create sequences
caption_seqs = tokenizer.texts_to_sequences([caption[0] for caption in captions])
caption_seqs = pad_sequences(caption_seqs, maxlen=MAX_LENGTH, padding='post')

print("Vocabulary size:", len(tokenizer.word_index))
print("Max caption length:", MAX_LENGTH)
print("Caption sequences:", caption_seqs)


Vocabulary size: 813
Max caption length: 143
Caption sequences: [[2 3 4 ... 0 0 0]
 [2 3 4 ... 0 0 0]
 [2 3 4 ... 0 0 0]
 ...
 [2 3 4 ... 0 0 0]
 [2 3 4 ... 0 0 0]
 [2 3 4 ... 0 0 0]]


In [10]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, RepeatVector, concatenate, Add, MultiHeadAttention, LayerNormalization, Conv1D

# Constants
IMG_SIZE = (299, 299)
VOCAB_SIZE = 5000  # Adjust based on your dataset
MAX_LENGTH = 128  # Max sequence length after padding
EMBED_DIM = 256
UNITS = 512

# Function to load and preprocess images
def load_image(image_path):
    img = load_img(image_path, target_size=IMG_SIZE)
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

# Function to load captions from a file
def load_captions(caption_file):
    with open(caption_file, 'r', encoding='utf-8') as f:
        captions = f.readlines()
    captions = [caption.strip().split('\t') for caption in captions]
    return captions

# Build InceptionV3 encoder
def build_encoder():
    base_model = InceptionV3(weights='imagenet')
    encoder = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)
    return encoder

# Build full image captioning model
def build_image_captioning_model(encoder, decoder):
    image_input = Input(shape=(299, 299, 3), name='image_input')
    encoded_image = encoder(image_input)
    caption_input = Input(shape=(MAX_LENGTH,), name='caption_input')
    output = decoder([encoded_image, caption_input])
    
    model = Model(inputs=[image_input, caption_input], outputs=output)
    return model

### Model 1: Transformer-based Model

# Different parameters for Model 1
EMBED_DIM_1 = 256
NUM_HEADS = 4
FF_DIM = 512

# Custom build_decoder function using Transformer
def build_transformer_decoder(vocab_size, embedding_dim, num_heads, ff_dim, max_length):
    # Image feature placeholder
    image_features = Input(shape=(2048,), name='image_features')
    
    # Captions as sequences
    inputs = Input(shape=(max_length,), name='input_captions')
    embeddings = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(inputs)
    embeddings = Dropout(0.5)(embeddings)

    # Repeat image features to match max_length
    image_features_repeated = RepeatVector(max_length)(image_features)
    
    # Transformer layer
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(embeddings, image_features_repeated)
    attn_output = Dropout(0.5)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(attn_output + embeddings)
    
    ffn_output = Dense(ff_dim, activation='relu')(out1)
    ffn_output = Dense(embedding_dim)(ffn_output)
    ffn_output = Dropout(0.5)(ffn_output)
    out2 = LayerNormalization(epsilon=1e-6)(ffn_output + out1)
    
    outputs = Dense(vocab_size, activation='softmax')(out2)
    model = Model(inputs=[image_features, inputs], outputs=outputs)
    return model

# Build encoder and decoder for Model 1
encoder_1 = build_encoder()
transformer_decoder = build_transformer_decoder(vocab_size=VOCAB_SIZE, embedding_dim=EMBED_DIM_1, num_heads=NUM_HEADS, ff_dim=FF_DIM, max_length=MAX_LENGTH)

# Build full image captioning model for Model 1
image_captioning_model_1 = build_image_captioning_model(encoder_1, transformer_decoder)

# Compile the model
image_captioning_model_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Print model summary
image_captioning_model_1.summary()

### Model 2: CNN-RNN Hybrid Model

# Different parameters for Model 2
EMBED_DIM_2 = 512
UNITS_2 = 1024

# Custom build_decoder function using CNN-RNN Hybrid
def build_cnn_rnn_decoder(vocab_size, embedding_dim, units, max_length):
    # Image feature placeholder
    image_features = Input(shape=(2048,), name='image_features')

    # Captions as sequences
    inputs = Input(shape=(max_length,), name='input_captions')
    embeddings = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(inputs)
    embeddings = Dropout(0.5)(embeddings)

    # Repeat image features to match max_length
    image_features_repeated = RepeatVector(max_length)(image_features)

    # Merge image features and caption embeddings
    merged = concatenate([image_features_repeated, embeddings])
    
    # CNN layers
    cnn_output = Conv1D(filters=units, kernel_size=3, padding='same', activation='relu')(merged)
    
    # RNN layer
    lstm = LSTM(units, return_sequences=True)(cnn_output)
    lstm = Dropout(0.5)(lstm)
    
    outputs = Dense(vocab_size, activation='softmax')(lstm)
    model = Model(inputs=[image_features, inputs], outputs=outputs)
    return model

# Build encoder and decoder for Model 2
encoder_2 = build_encoder()
cnn_rnn_decoder = build_cnn_rnn_decoder(vocab_size=VOCAB_SIZE, embedding_dim=EMBED_DIM_2, units=UNITS_2, max_length=MAX_LENGTH)

# Build full image captioning model for Model 2
image_captioning_model_2 = build_image_captioning_model(encoder_2, cnn_rnn_decoder)

# Compile the model
image_captioning_model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Print model summary
image_captioning_model_2.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 model_1 (Functional)        (None, 2048)                 2180278   ['image_input[0][0]']         
                                                          4                                       
                                                                                                  
 caption_input (InputLayer)  [(None, 128)]                0         []                            
                                                                                                  
 model_2 (Functional)        (None, 128, 5000)            7550856   ['model_1[0][0]',       

In [11]:
# Save the models
image_captioning_model_1.save('image_captioning_model_1.h5')
image_captioning_model_2.save('image_captioning_model_2.h5')


C:\Users\admin\Downloads\Project Formats\opo\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=IMG_SIZE)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img


In [26]:
# Load the models
model_1 = tf.keras.models.load_model('image_captioning_model_1.h5', custom_objects={'MultiHeadAttention': tf.keras.layers.MultiHeadAttention, 'LayerNormalization': tf.keras.layers.LayerNormalization})
model_2 = tf.keras.models.load_model('image_captioning_model_2.h5', custom_objects={'Conv1D': tf.keras.layers.Conv1D})
